# APLICACIONES EN CIENCIAS DE COMPUTACION

## Laboratorio 4:  Búsqueda Local (Hill climbing, Simulated annealing y Beam search) 
Indicaciones previas:
- Las respuestas deben tener un buen fundamento teórico, se realizarán descuentos en el puntaje a respuestas que no contesten a lo solicitado
- Cualquier indicio de plagio resultará en la anulación de la prueba.

La tarea de este laboratorio consiste en comparar métodos de busqueda local para la resolución de la N-Reinas.<br>Al final de este notebook se encuentran las preguntas que serán evaluadas en este laboratorio. 

Ejemplo de representacion de estado (tablero nqueens):
<img src="state_nqueens.png">


In [2]:
import numpy as np
from collections import defaultdict
from collections import Counter
from random import randrange
import random
from time import time
from random import shuffle, random, sample, randint, choice, uniform
from copy import deepcopy
import math
from math import exp
import sys

### Clase <b>SearchProblem</b>

Esta es una clase abstracta para definir problemas de busqueda. Se debe hacer subclases que implementen los metodos de las acciones, resultados, test de objetivo y el costo de camino. Entonces se puede instanciar las subclases y resolverlos con varias funciones de busqueda.

In [3]:
class SearchProblem:
    def __init__(self, initial=None):
        """Initialize a search problem with a initial state"""
        pass

    def initial(self):
        """Return default initial state of the search problem"""
        pass

    def value(self, state):
        """Return the value of the state. This is the objective function to be optimized"""
        pass

    def neighborhood(self, state):
        """Return the neighboring states of the given state"""
        pass

    def random_neighbor(self, state):
        """Return a random neighbor of the neighborhood of the state (used in simulated annealing)"""
        return choice(self.neighborhood(state))



### Clase <b>NQueensSearch</b>

La clase NQueenSearch implementa concretamente el problema del  tablero de las NQueens. Esta se representa mediante una tupla, en la cual se indica la posición de cada reina Q. Además, incluye el metodo value() para conocer la cantidad de pares de reinas atacadas mutuamente.

In [5]:
class NQueensSearch(SearchProblem):
    '''
    State: (QueenCoords)
    '''
    def __init__(self, filename,N=8):
        self.N = N
        self.file = filename

    def initial(self):
        # Lee el archivo para retornar una tupla con las posiciones de cada Reinas del tablero
        fd=open(self.file,"r+")
        puzzle=eval(fd.readline())
        board=[]
        for i in puzzle:
            board.append(i)
        return tuple(board)
    
    def value(self, state):
        """
        Retorna número de pares de Queens que se atacan mutuamente. Se recorre State: (QueenCoords) 
        para agregar el ataque de cada reina, tanto en sus diagonales como en su misma columna. Luego, 
        se recorre cada collection creado para incrementar el número de pares de reinas atacadas (clashes)
        """
        columnQ, diag1Q, diag2Q = [Counter() for i in range(3)]
        # En este caso state = (6,0,6,5,4,6,5,0) 
        for row, col in enumerate(state):
            columnQ[col] += 1       
            diag1Q[row - col] += 1  
            diag2Q[row + col] += 1
        clashes = 0
        for cnt in [columnQ, diag1Q, diag2Q]:
            for key in cnt:
                clashes += cnt[key] * (cnt[key] - 1) // 2
        return clashes

    def neighborhood(self, state):
        # Crea nuevos tableros vecinos, diferentes al original
        neighborhood = []
        for row in range(self.N):  # por cada fila
            for col in range(self.N):  # por cada columna
                # genera tablero vecino moviendo reyna de fila row a la columna col (siempre que no exita reyna en (row,col))    
                if col != state[row]: 
                    neighbor = list(state)
                    neighbor[row] = col
                    neighborhood.append(tuple(neighbor))
        return neighborhood

    def random_neighbor(self, state):
        # Genera un tablero vecino de manera aleatoria, a partir del tablero original pasado (state)
        row = randrange(self.N)  # escoge una fila aleatoriamente
        col = choice( [i for i in range(self.N) if i!=state[row]] ) # escoge una columna aleatoria diferente de donde esta la reyna en esa fila
        neighbor = list(state)
        neighbor[row] = col
        return tuple(neighbor)

### Funciones utilitarias para manejar el tablero NQueens</b>
Estas son funciones utilitarias para mostrar el tablero 

In [7]:
n = 8
def make_board(result):
    board = []
    espacio =['_']*(n + 1)
    espacio[0]=' '
    board.append(str().join(espacio))
    for col in result:
        line = ['*'] * (n + 2)
        line[0]='|'
        line[col + 1] = 'Q'
        line[n + 1]='|'
        board.append(str().join(line))
    board.append(str().join(espacio))
    return board

def print_board(board):
    charlist = list(map(list, board))
    for line in charlist:
        print(" ".join(line))
def show_board(current):
    # Muestra la distribución de Queens(Q) en el tablero.
    board = make_board(current)
    printBoard(board)

## <b>Algoritmos de Búsqueda Local</b> 

### <b>Hill-climbing </b> 

Implementación del algoritmo para la resolución de NQueenSearchs

In [9]:
def hill_climbing(problem, max_iter=1000):
    count = 0  # contador de iteraciones desde que se encuentra el 1er tablero solucion 
    current = problem.initial()  # lee el archivo del tablero inicial
    current_score = problem.value(current)  # evalua tablero inicial
    
    # Muestra tablero inicial
    print('Hill Climbing intentará resolver el siguiente tablero NQueens:')
    show_board(current)  
    print()
        
    start_time = time()  # inicia el contador de tiempo
    t = 0
    while (t < max_iter):
        if (t % 100 == 0): 
            print('Iteration {},\tCurrent score  = {}'.format(t, problem.value(current)))
            
        neighborhood = problem.neighborhood(current)
        if not neighborhood:
            break
            
        neighborhood_scores = []
        for i in range(len(neighborhood)): # evalua cada tablero vecino
            neighborhood_scores.append(problem.value(neighborhood[i]) )
        index_best_neighbor = np.argmin(neighborhood_scores)  # obtiene el indice del mejor tablero
        
       
        if neighborhood_scores[index_best_neighbor] <= current_score:  # si el mejor vecino es mejor que el tablero current
            current_score = neighborhood_scores[index_best_neighbor]
            current = deepcopy(neighborhood[index_best_neighbor])
        
        if problem.value(current) == 0:  # si es tablero solucion
            count += 1  # aumenta contador de tableros solucion encontrados
        
        t += 1
    end_time = time()  # stop el contador de tiempo
    print('\nN° de tableros solución: %2d en %d iteraciones \nRunning time: %f'%(count,maxIter , end_time-start_time))
    print('Mejor tablero solución hallado con score {}'.format(problem.value(current)))
    show_board(current)  # muestra el tablero final


### <b>Simulating Annealing</b> 
Implementación del algoritmo para la resolución de NQueenSearchs

In [10]:
def simulated_annealing(problem, t0, dr, max_iter):
    """
    Simulating annealing solver.
    t0: Initial temperature
    dr: The decay rate of the schedule function: Ti = T0*(DR)^i (Ti is the temperature at iteration i). 
    For efficiecy the schedule function is implemented as: T_i = T_{i - 1} * DR.
    max_iter: The maximum number of iterations
    """
    count = 0  # contador de iteraciones desde que se encuentra el 1er tablero solucion 
    current = problem.initial()  # lee el archivo del tablero inicial
    current_score = problem.value(current)    # evalua tablero inicial
   
    # muestra tablero inicial    
    print('Simulated Annealing intentará resolver el siguiente tablero NQueens: ')
    show_board(current)  # muestra tablero inicial
    print()    
    
    start_time = time()  # inicia el contador de tiempo
    best_score = current_score
    T = t0  # inicia temperatura en temperatura inicial
    t = 0
    while (t < max_iter):
        if (t % 10000 == 0): 
            print('Iteration {},\tTemperaure = {},\tBest score = {},\tCurrent score = {}'.format(t, T,best_score, current_score))

        neighbor = problem.random_neighbor(current)
        neighbor_score = problem.value(neighbor)  # evalua tablero vecino
        delta = float(neighbor_score - current_score)  # diferencia entre el score del tablero vecino con respecto al actual
            
        if (delta < 0):   # si el tablero vecino generado es mejor que el actual, se acepta
            current = neighbor
            current_score = neighbor_score 
        elif (random() < exp(abs(delta) / T)):  # si el tablero generado es peor, se acepta con probability  exp((delta/T))
            current = neighbor
            current_score = neighbor_score 
            
        if (current_score  < best_score):  # si el tablero actual es mejor que el mejor tablero encontado hasta ahora
            best_board = deepcopy(current)
            best_score = current_score
                
        if current_score == 0:  # si es tablero solucion   
            best_board = current
            best_score = current_score
            if count == 0: 
                best_iteration = t   # iteracion donde se encontro el 1er tablero solucion
            count += 1  # aumenta contador de tableros solucion encontrados
            
        T = T * dr   # aplica decaimiento de temperatura
        t += 1
    end_time = time()  # stop del contador de tiempo
    
    if best_score == 0:
        print ("\nSA encontro tablero solucion en iteracion = {} de {} iteraciones".format(best_iteration, t))
    else:
        print("\nSA no encontró tablero solucion!. Este es el mejor tablero encontrado con score={}:".format(best_score))
    
    print("N° de tableros solución: %2d en %d iteraciones \nRunning time: %f"%(count, t , end_time - start_time))
   
    show_board(best_board)  # muestra el mejor tablero

## <b> Experimentación con los algoritmos de Búsqueda</b> 

In [14]:
# ESTA CELDA NO NECESITA SER MODIFICADA
""" Carga un tablero de archivo en disco e instancia el problema de busqueda  """
problem = NQueensSearch("queens.txt")
print("8-Queens Original")
ShowBoard(problem.initial())

8-Queens Original
  _ _ _ _ _ _ _ _
| * * * * * * Q * |
| Q * * * * * * * |
| * * * * * * Q * |
| * * * * * Q * * |
| * * * * Q * * * |
| * * * * * * Q * |
| * * * * * Q * * |
| Q * * * * * * * |
  _ _ _ _ _ _ _ _


### Hill Climbing

Llama a Hill Climbing para resolver el tablero 'queens.txt' con numero de iteraciones maxIter=1000

In [23]:
problem = NQueensSearch("queens.txt")
hill_climbing(problem, 1000)

Hill Climbing intentará resolver el siguiente tablero NQueens:
  _ _ _ _ _ _ _ _
| * * * * * * Q * |
| Q * * * * * * * |
| * * * * * * Q * |
| * * * * * Q * * |
| * * * * Q * * * |
| * * * * * * Q * |
| * * * * * Q * * |
| Q * * * * * * * |
  _ _ _ _ _ _ _ _

Iteration 0,	Current score  = 10
Iteration 100,	Current score  = 1
Iteration 200,	Current score  = 1
Iteration 300,	Current score  = 1
Iteration 400,	Current score  = 1
Iteration 500,	Current score  = 1
Iteration 600,	Current score  = 1
Iteration 700,	Current score  = 1
Iteration 800,	Current score  = 1
Iteration 900,	Current score  = 1

N° de tableros solución:  0 en 1000 iteraciones 
Running time: 2.438860
Mejor tablero solución hallado con score 1
  _ _ _ _ _ _ _ _
| * * Q * * * * * |
| * * * * * Q * * |
| * * * * * * * Q |
| * Q * * * * * * |
| * * * * Q * * * |
| * * * * * * Q * |
| * * * Q * * * * |
| Q * * * * * * * |
  _ _ _ _ _ _ _ _


### Simulated Annealing

In [29]:
problem = NQueensSearch("queens.txt")
simulated_annealing(problem, 0.5, .99999, 100000)

Simulated Annealing intentará resolver el siguiente tablero NQueens: 
  _ _ _ _ _ _ _ _
| * * * * * * Q * |
| Q * * * * * * * |
| * * * * * * Q * |
| * * * * * Q * * |
| * * * * Q * * * |
| * * * * * * Q * |
| * * * * * Q * * |
| Q * * * * * * * |
  _ _ _ _ _ _ _ _

Iteration 0,	Temperaure = 0.5,	Best score = 10,	Current score = 10
Iteration 10000,	Temperaure = 0.45241848280737684,	Best score = 1,	Current score = 6
Iteration 20000,	Temperaure = 0.4093649671714617,	Best score = 1,	Current score = 9
Iteration 30000,	Temperaure = 0.3704085547244124,	Best score = 1,	Current score = 7
Iteration 40000,	Temperaure = 0.33515935269458563,	Best score = 1,	Current score = 7
Iteration 50000,	Temperaure = 0.3032645716895745,	Best score = 1,	Current score = 10
Iteration 60000,	Temperaure = 0.2744049948260552,	Best score = 1,	Current score = 8
Iteration 70000,	Temperaure = 0.24829178286794154,	Best score = 1,	Current score = 10
Iteration 80000,	Temperaure = 0.22466358339730758,	Best score = 0,	Curren

# Preguntas:
**1.** Se presenta el tablero: 'queens.txt' de las 8-Queens con función de costo: 
    
        h = número de pares de reinas que se atacan mutuamente    

Además, se implementan los algoritmos Simulating Annealing (SA) y Hill Climbing (HC) con los siguientes parámetros (ellos garantizan una misma cantidad de tableros evaluados como máximo):

        HC: maxIter=1000
        SA: T0=0.5, DR=.99999, maxIter=100000 
        
En el presente laboratorio, se proponen los algoritmos de búsqueda local, los cuales **maximizan** la función. Se solicita modificar el código en ambos algoritmos, con la finalidad de **minimizar** la función de costo (h) **(4 pts)**

Completado.

**2.** Después de haber completado el código, ¿el algoritmo Hill Climbing presenta soluciones óptimas? ¿Cuáles son las limitaciones que puede presentar este algoritmo de búsqueda local, según los resultados? **(4 pts)**

El algoritmo **Hill Climbing** no presentó soluciones óptimas en nuestro caso particular y en un caso general tampoco tendría por qué hacerlo ya que realiza una estrategia greedy de moverse en dirección del mejor estado vecino y puede que nos quedemos estancados en un mínimo local.
A partir de los resultados obtenidos, podemos observar que en la iteración 100 ya habíamos llegado a un tablero con score 1, lo cual quiere decir que hasta las 1000 iteraciones hemos o bien hemos encontrado peores tableros u otros con el mismo valor. Las limitaciones de este algoritmo es que si nos hemos estancado en un mínimo local, es decir, en este caso, si nuestro tablero o algunos vecinos tienen score 1 y sus vecinos mayores a este, para poder llegar a una solución deberíamos salir de esta depresión en la que nos encontramos, sin embargo, como fue implementado el algoritmo no podemos realizarlo.

**3.** En cuanto a las soluciones encontradas por Simulated Annealing, ¿este algoritmo presenta soluciones óptimas? ¿Cómo se pueden interpretar y relacionar los resultados con su teoría y propiedades? Por otro lado, ¿cómo controlamos el grado de exploración de un algoritmo Simulated Annealing? **(4 pts)**

En mis resultados pude conseguir una solución óptima; no obstante, como hay un factor de aleatoriedad a la hora de aceptar errores, si corremos varias veces mi test es posible que encontremos algunos casos en donde nos quedamos en un mínimo local.
Teóricamente, hay un teorema que nos dice que si nuestra temperatura decrece exponencialmente lento, entonces vamos a converger a un estado óptimo. Un problema es que una lenta convergencia no implica que en práctica pueda ser computable rápidamente pero al menos nos garantiza convergencia.
En mis resultados podemos observar que en las 10 000 primera iteraciones, me encontraba en un estado con valor 6; y luego observamos que para las 20 000 iteraciones me encontraba en ese momento en uno con valor 9, mientras que en ambas mi mejor score hasta ese momento fue igual a 1. Esto quiere decir que nunca pudo llegar hasta ese momento a algún tablero con score 0 y no tuvo de otra que aceptar errores y por eso que en determinados momentos lo encontramos con valores de estados más grandes. Observamos que en las 80 000 iteraciones, logró encontrar alguna solución óptima con score 0 y como quiere seguir mejorando esto, sigue buscando otros estados y aparece en uno con valor 7. Lo bueno es que a pesar de seguir buscando más soluciones, a diferencia del algoritmo **Hill Climbing**, este proceso de aceptar errores nos permitió encontrar un mínimo global luego de casi 70 000 operaciones.
Finalmente, el grado de exploración del algoritmo es controlado por la temperatura, conforme más altas temperaturas tenemos, más pasos localmente y aparentemente malos podremos aceptar, mientras que una temperatura más baja implica que aceptaremos menos soluciones no tan buenas localmente, adicionamos a esto la convergencia explicada en el párrafo anterior cuando decrementa de manera lenta.

**4.** Usando fundamento teórico, ¿cuáles son las principales ventajas de Simulated Annealing sobre Hill Climbing? Además, relacione su respuesta con los resultados obtenidos en las pruebas. **(4 pts)**

Lo primero que podemos argumentar sin entrar tanto a detalla es que **Hill Climbing** sería un caso particular de **Simulated Annealing** cuando nuestra temperatura es siempre cero, puesto que aceptaremos todo lo que mejore y rechazaremos siempre todo lo que no mejore. De esta manera, si es que **Hill Climbing** nos ofrece alguna ventaja, entonces **Simulated Annealing** también y cualquier cosa adicional será a favor de esta última.
Ahora podemos entrar a detalles, como respondo en las limitaciones mencionadas en la **pregunta 2** y una estrategia para mejorar **Hill Climbing** en la **pregunta 5**, el hecho de tener un factor aleatorio que nos permita aceptar movimientos no óptimos nos puede permitir escapar de quedarnos atrapados en un mínimo local.

**5.** ¿Qué estrategias de mejora se pueden aplicar en Hill Climbing, en el caso de NQueens? **(2 pts)**

Una estrategia que podríamos utilizar en nuestro caso particular, cuando en nuestro conjunto de estados hemos estado obteniendo continuamente resultados que no nos permiten mejorar, podríamos haber realizado ciertos movimientos de manera aleatoria para ver qué pasaría si no sigo el greedy.
En caso siga estancado alrededor de valores que no mejoran mi mejor valor, o bien hay demasiados casos por analizar, o lo más probable es que nuestro algoritmo converge a un mínimo local.

**6.** Justificar, teóricamente, la limitación del método de búsqueda Beam search **(2 pts)**

La principal limitación del método **Beam Search** es, como mencionan los autores de la bibliografía y el profesor durante clase, la falta de diversidad. Esto quiere decir que puede que se encuentre el conjunto solución concentrado en alguna región y que todos nuestros algoritmos se comuniquen y decidan ir hacia dicha región, lo cual implicaría que estamos gastando más recursos computacionales para una situación quizás un poco más sencilla. Para ejemplificar mejor, sea $k$ el número de algoritmos **Local Search** que nuestro **Beam Search** correrá en paralelo, si es que tenemos la solución en alguno de los algoritmos que están corriendo y hubiese sido necesario solo analizar en ese, esto nos cuesta en espacio y tiempo $k$ veces más frente a lo que hubiese costado realizar una simple búsqueda local con algoritmos vistos en este laboratorio.
Para lidiar con estas limitaciones, una manera estocástica de lidiar con esto es escoger los mejores $k$.